In [114]:
import pandas as pd
import numpy as np

In [115]:
data = pd.read_csv('All_Sets_Resume_Data_1.csv')

In [116]:
data.head()

,Unnamed: 0,name,cities,final_technologies,Company_Latest,CompanyO1,CompanyO2,Tier_Latest,Tier_Old1,Tier_Old2
0,0,madhavi meka java (1).docx,hyderabad,java,tcs,not specified,not specified,Tier-1,not specified,not specified
1,1,lakshmi_narayan.palla.docx,not specified,others,mindtree,not specified,not specified,Tier-2,not specified,not specified
2,2,vineet.docx,gurgaon,"java, pega",accenture,mphasis,not specified,Tier-1,Tier-2,not specified
3,3,t venkatesh kumar- a.docx,bengaluru,"java, python",jio platform ajio,not specified,not specified,Tier-3,not specified,not specified
4,4,resume - jun'23 (1).docx,not specified,others,british petroleum,not specified,not specified,Tier-3,not specified,not specified


In [117]:
# !pip install pelote
technologies = ['java, pega']
if technologies:
    # Create an empty list to store filtered DataFrames
    filtered_datas = []
    for technology in technologies:
        # Filter for each technology and append the result to the list
        filtered_data = data[data["final_technologies"].eq(technology)]
        filtered_datas.append(filtered_data)
    # Combine all filtered DataFrames using bitwise OR (|)
    df_filtered = pd.concat(filtered_datas, ignore_index=True)
else:
    # No technologies provided, keep the entire DataFrame
    df_filtered = data.copy()



In [118]:
df_filtered

,Unnamed: 0,name,cities,final_technologies,Company_Latest,CompanyO1,CompanyO2,Tier_Latest,Tier_Old1,Tier_Old2
0,2,vineet.docx,gurgaon,"java, pega",accenture,mphasis,not specified,Tier-1,Tier-2,not specified
1,45,kiranmaiee kalidindi-pega.docx,bengaluru,"java, pega",mphasis,blue rose,accenture,Tier-2,Tier-3,Tier-1
2,69,naveen kumar boda.docx,hyderabad,"java, pega",mphasis,not specified,not specified,Tier-2,not specified,not specified
3,96,abdul baseer.docx,hyderabad,"java, pega",clinztech research,not specified,not specified,Tier-3,not specified,not specified
4,114,gaurav sadashiv londhe.docx,not specified,"java, pega",accenture,not specified,not specified,Tier-1,not specified,not specified
...,...,...,...,...,...,...,...,...,...,...
57,5111,krishnamohanneethipudi-mainframe lead.pdf,not specified,"java, pega",ciber sites,not specified,not specified,Tier-3,not specified,not specified
58,5188,mallikarjun.pdf,not specified,"java, pega",inosoft datalinks,not specified,not specified,Tier-3,not specified,not specified
59,5511,naukri_jimmysaraf[7y_8m].pdf,not specified,"java, pega",highradius,not specified,not specified,Tier-3,not specified,not specified
60,5787,naukri_rvvsatyanarayanaprasad[8y_0m].pdf,hyderabad,"java, pega",united health,not specified,not specified,Tier-3,not specified,not specified


In [119]:
df_filtered['final_technologies'].unique()

array(['java, pega'], dtype=object)

In [120]:
# df=df.replace('not specified', np.nan, inplace=True)
unique_companies_0 = df_filtered["Company_Latest"].unique().tolist()
unique_companies_1 = df_filtered["CompanyO1"].unique().tolist()
unique_companies_2 = df_filtered["CompanyO2"].unique().tolist()
all_companies = unique_companies_0 + unique_companies_1 + unique_companies_2

dict_movements = df_filtered.drop(["name", "cities", "final_technologies"], axis=1).to_dict(
    orient="records"
)

In [121]:
latest_movement = []
previous_movement = []

for i in range(len(dict_movements)):
    current_dict = dict_movements[i]
    latest_movement.append(
        tuple([current_dict["CompanyO1"], current_dict["Company_Latest"]])
    )
    previous_movement.append(
        tuple([current_dict["CompanyO2"], current_dict["CompanyO1"]])
    )

In [122]:
latest_movement_df = pd.DataFrame(latest_movement, columns=["from", "to"])
previous_movement_df = pd.DataFrame(previous_movement, columns=["from", "to"])
previous_movement_df.replace("not specified", np.nan, inplace=True)
latest_movement_df.replace("not specified", np.nan, inplace=True)
latest_movement_df.dropna(inplace=True)
previous_movement_df.dropna(inplace=True)
latest_movements_list = list(latest_movement_df.to_records(index=False))
previous_movements_list = list(previous_movement_df.to_records(index=False))

In [123]:
count_df_latest = pd.DataFrame(
        latest_movement_df["to"].value_counts()
    ).reset_index()
count_df_previous = pd.DataFrame(
    previous_movement_df["to"].value_counts()
).reset_index()
combined_count_df = pd.merge(
    count_df_latest, count_df_previous, on="to", how="outer"
)
combined_count_df.replace(np.nan, 0, inplace=True)
combined_count_df["total_to"] = (
    combined_count_df["count_x"] + combined_count_df["count_y"]
)
# combined_count_df1 = combined_count_df
combined_df = pd.concat([previous_movement_df,latest_movement_df],axis=0)
# print(combined_df)
combined_count_dict = (
    combined_count_df.drop(columns=["count_x", "count_y"])
    .set_index("to")["total_to"]
    .to_dict()
)


In [124]:
latest_movements_list

[('mphasis', 'accenture'),
 ('blue rose', 'mphasis'),
 ('capgemini', 'syntel'),
 ('optum', 'coforge'),
 ('legato health', 'mindtree'),
 ('cognizant', 'coforge'),
 ('capgemini', 'amex'),
 ('virtusa', 'tcs'),
 ('wells fargo', 'tech mahindra'),
 ('business intelli', 'hcl'),
 ('stratsol', 'evoke'),
 ('ltimindtree', 'wipro'),
 ('brandwidth', 'atos syntel'),
 ('accenture', 'cognizant  '),
 ('dll de lage landen financial', 'cognizant'),
 ('ciberspring', 'thryve digital'),
 ('coforge', 'evoke'),
 ('glovision techno', 'tcs'),
 ('mphasis', 'accenture'),
 ('accenture', 'mindtree'),
 ('ltimindtree', 'wipro'),
 ('areteans  ', 'standard chartered'),
 ('coforge', 'mindtree'),
 ('infosys', 'santechsoft'),
 ('coforge ', 'accenture '),
 ('uhc', 'coforge'),
 ('link group', 'wells fargo'),
 ('virtusa    ', 'mks vision  '),
 ('serendebyte   ', 'unify   '),
 ('spotifin marketplace', 'sandhata')]

In [125]:
previous_movements_list

[('accenture', 'blue rose'),
 ('zensar', 'capgemini'),
 ('genpact', 'optum'),
 ('infosys', 'legato health'),
 ('tech mahindra', 'cognizant'),
 ('versatile', 'capgemini'),
 ('coforge dpa', 'virtusa'),
 ('link group', 'wells fargo'),
 ('stu biz solution', 'stratsol'),
 ('bangalore', 'ltimindtree'),
 ('inc', 'dll de lage landen financial'),
 ('verizon', 'ciberspring'),
 ('infosys', 'coforge'),
 ('cibc', 'glovision techno'),
 ('zensar', 'accenture'),
 ('tech mahindra', 'ltimindtree'),
 ('w3softech', 'areteans  '),
 ('synechron', 'coforge'),
 ('tcs', 'infosys'),
 ('suntec business ', 'coforge '),
 ('optum', 'uhc'),
 ('srin soft tech', 'link group'),
 ('ciber sites   ', 'virtusa    '),
 ('arctern  ', 'serendebyte   '),
 ('tcs', 'spotifin marketplace')]

In [126]:
col1_values = combined_df['from'].unique().tolist()
col2_values = combined_df['to'].unique().tolist()

from_values = list(set(col1_values))
to_values = list(set(col2_values))
combined_list = from_values + to_values
combined_list1 = list(set(combined_list))

In [127]:
latest_movement_df

,from,to
0,mphasis,accenture
1,blue rose,mphasis
8,capgemini,syntel
9,optum,coforge
12,legato health,mindtree
13,cognizant,coforge
14,capgemini,amex
15,virtusa,tcs
18,wells fargo,tech mahindra
19,business intelli,hcl


In [128]:
previous_movement_df

,from,to
1,accenture,blue rose
8,zensar,capgemini
9,genpact,optum
12,infosys,legato health
13,tech mahindra,cognizant
14,versatile,capgemini
15,coforge dpa,virtusa
18,link group,wells fargo
20,stu biz solution,stratsol
21,bangalore,ltimindtree


In [129]:
print(latest_movement_df.shape)
print(previous_movement_df.shape)

(30, 2)
(25, 2)


In [143]:
previous_movement_df['type'] = 'PM'
latest_movement_df['type'] = 'LM'

In [144]:
latest_movement_df.head()

,from,to,type
0,mphasis,accenture,LM
1,blue rose,mphasis,LM
8,capgemini,syntel,LM
9,optum,coforge,LM
12,legato health,mindtree,LM


In [145]:
previous_movement_df.head()

,from,to,type
1,accenture,blue rose,PM
8,zensar,capgemini,PM
9,genpact,optum,PM
12,infosys,legato health,PM
13,tech mahindra,cognizant,PM


In [147]:
final_df = pd.concat([latest_movement_df,previous_movement_df],axis=0)
final_df.reset_index(drop=True,inplace=True)
final_df

,from,to,type
0,mphasis,accenture,LM
1,blue rose,mphasis,LM
2,capgemini,syntel,LM
3,optum,coforge,LM
4,legato health,mindtree,LM
5,cognizant,coforge,LM
6,capgemini,amex,LM
7,virtusa,tcs,LM
8,wells fargo,tech mahindra,LM
9,business intelli,hcl,LM


In [152]:
final_df.dtypes

from    object
to      object
type    object
dtype: object

In [130]:
# for i in latest_movements_list:
#         net.add_edge(
#             i[0], i[1], color="#F2613F", physics=False, title=i[0] + " to " + i[1]
#         )
#     for i in previous_movements_list:
#         net.add_edge(
#             i[0], i[1], color="White", physics=False, title=i[0] + " to " + i[1]
#         )

In [148]:
import networkx as nx

In [153]:
from pelote import edges_table_to_graph
g1 = edges_table_to_graph(final_df,edge_source_col="from",edge_target_col="to",edge_data=["type"],count_rows_as_weight=True,directed=True)


In [154]:
# # Compute in-degree and out-degree for each node

# in_degrees = dict(g.in_degree())
# out_degrees = dict(g.out_degree())

# print(g.nodes)
# print(g.edges)
# # print(out_degrees)
# # print(in_degrees)

In [134]:
# def edge_color_func(g,out_degrees,in_degrees):
#     for node in g.nodes:
#         source_in_degree = in_degrees[node]
#         source_out_degree = out_degrees[node]
#         if source_in_degree > 0:
#             return 'green'
#         elif source_out_degree > 0 :
#             return 'red'

In [155]:
from ipysigma import Sigma
sigma1= Sigma(g1,edge_color='green')
sigma1

Sigma(nx.DiGraph with 57 nodes and 52 edges)

In [142]:
sigma2= Sigma(g2,edge_color='red')
sigma2

Sigma(nx.DiGraph with 42 nodes and 28 edges)

In [140]:
# Combine the two Sigma widgets
sigma2 = sigma1 + sigma2

TypeError: unsupported operand type(s) for +: 'Sigma' and 'Sigma'